# [610. Triangle Judgement](https://leetcode.com/problems/triangle-judgement/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Triangle

<pre>+-------------+------+
| Column Name | Type |
+-------------+------+
| x           | int  |
| y           | int  |
| z           | int  |
+-------------+------+</pre>
In SQL, (x, y, z) is the primary key column for this table.
Each row of this table contains the lengths of three line segments.


Report for every three line segments whether they can form a triangle.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Triangle table:
<pre>+----+----+----+
| x  | y  | z  |
+----+----+----+
| 13 | 15 | 30 |
| 10 | 20 | 15 |
+----+----+----+</pre>
Output:
<pre>+----+----+----+----------+
| x  | y  | z  | triangle |
+----+----+----+----------+
| 13 | 15 | 30 | No       |
| 10 | 20 | 15 | Yes      |
+----+----+----+----------+</pre>

In [1]:
# pandas schema

import pandas as pd

data = [[13, 15, 30], [10, 20, 15]]
triangle = pd.DataFrame(data, columns=['x', 'y', 'z']).astype({'x': 'Int64', 'y': 'Int64', 'z': 'Int64'})

# Converting to pyspask schema

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

triangle_df = spark.createDataFrame(triangle)
triangle_df.show()

+---+---+---+
|  x|  y|  z|
+---+---+---+
| 13| 15| 30|
| 10| 20| 15|
+---+---+---+



In [2]:
#in dataFrame

triangle_df.withColumn('triangle',when( (col('x')+col('y') > col('z')) & (col('z')+col('y') > col('x')) & (col('x')+col('z') > col('y')),'Yes').otherwise('No'))\
           .show()

+---+---+---+--------+
|  x|  y|  z|triangle|
+---+---+---+--------+
| 13| 15| 30|      No|
| 10| 20| 15|     Yes|
+---+---+---+--------+



In [3]:
#using expr
triangle_df.withColumn('triangle',expr("case when (x+y) > z and (x+z) > y and (z+y) > x then 'Yes' else 'No' end"))\
           .show()

+---+---+---+--------+
|  x|  y|  z|triangle|
+---+---+---+--------+
| 13| 15| 30|      No|
| 10| 20| 15|     Yes|
+---+---+---+--------+



In [4]:
# In spark SQL

triangle_df.createOrReplaceTempView('triangle')

spark.sql('''
select *,
case
    when (x+y) > z and (x+z) > y and (z+y) > x then 'Yes' else 'No'
end as triangle
from Triangle
''').show()

+---+---+---+--------+
|  x|  y|  z|triangle|
+---+---+---+--------+
| 13| 15| 30|      No|
| 10| 20| 15|     Yes|
+---+---+---+--------+



In [5]:
# in udf pyspark

from pyspark.sql.types import StringType


def is_triangle(x, y, z):
    if x + y > z and x + z > y and y + z > x:
        return "Yes"
    return "No"


udf_is_triangle = udf(is_triangle, StringType())

triangle_df.withColumn('triangle', udf_is_triangle('x','y','z')).show()

+---+---+---+--------+
|  x|  y|  z|triangle|
+---+---+---+--------+
| 13| 15| 30|      No|
| 10| 20| 15|     Yes|
+---+---+---+--------+



In [7]:
# In spark SQL

triangle_df.createOrReplaceTempView('triangle')
spark.udf.register('udf_is_triangle',is_triangle)

spark.sql('''
select x,y,z, udf_is_triangle(x,y,z) as triangle from triangle;
''').show()

+---+---+---+--------+
|  x|  y|  z|triangle|
+---+---+---+--------+
| 13| 15| 30|      No|
| 10| 20| 15|     Yes|
+---+---+---+--------+



In [ ]:
spark.stop()